In [1]:
import os
import torch
import torchvision
import numpy as np
import foolbox.ext.native as fbn
import eagerpy as ep

import transforms
import resnet_3layer as resnet

In [2]:
num_sample_MIOL = 15
lamdaOL = 0.6

### Load backbone model

In [3]:
CLASSIFIER = resnet.model_dict['resnet50']
classifier = CLASSIFIER(num_classes=10)

device = torch.device("cuda:0")
classifier = classifier.to(device)

classifier.load_state_dict(torch.load('model_IAT.ckpt'))
classifier.eval();

### Construct image pools

In [4]:
def onehot(ind):
    vector = np.zeros([10])
    vector[ind] = 1
    return vector.astype(np.float32)

train_trans, test_trans = transforms.cifar_transform()
trainset = torchvision.datasets.CIFAR10(root='~/cifar/',
                                        train=False,
                                        download=True,
                                        transform=train_trans,
                                        target_transform=onehot)
testset = torchvision.datasets.CIFAR10(root='~/cifar/',
                                       train=False,
                                       download=True,
                                       transform=test_trans,
                                       target_transform=onehot)

dataloader_train = torch.utils.data.DataLoader(
    trainset,
    batch_size=1,
    shuffle=True,
    num_workers=2)

dataloader_test = torch.utils.data.DataLoader(
    testset,
    batch_size=10,
    shuffle=False,
    num_workers=5)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
from tqdm import tqdm
num_pool = 10000
mixup_pool_OL = {}

for i in range(10):
    mixup_pool_OL.update({i: []})

for i, data_batch in tqdm(enumerate(dataloader_train), total=num_pool):
    img_batch, label_batch = data_batch
    img_batch = img_batch.to(device)
    _, label_ind = torch.max(label_batch.data, 1)
    mixup_pool_OL[label_ind.numpy()[0]].append(img_batch)
    if i >= (num_pool - 1):
        break

print('Finish constructing mixup_pool_OL')

 99%|█████████▉| 9914/10000 [00:11<00:00, 880.24it/s] 

Finish constructing mixup_pool_OL


### Construct surrogate models that wrap OL within model

In [6]:
import torch.nn as nn
import torch.nn.functional as F

soft_max = nn.Softmax(dim=-1)

class CombinedModel(nn.Module):
    def __init__(self, classifier):
        super(CombinedModel, self).__init__()
        self.classifier = classifier

    def forward(self, img_batch):
        pred_cle_mixup_all_OL = 0 # torch.Tensor([0.]*10)
        
        # forward pass without PL/OL
        pred_cle = self.classifier(img_batch)
        cle_con, predicted_cle = torch.max(soft_max(pred_cle.data), 1)
        predicted_cle = predicted_cle.cpu().numpy()
            
        # perform MI-OL
        for k in range(num_sample_MIOL):
            mixup_img_batch = np.empty(img_batch.shape, dtype=np.float32)
            
            for b in range(img_batch.shape[0]):
                # CLEAN
                xs_cle_label = np.random.randint(10)
                while xs_cle_label == predicted_cle[b]:
                    xs_cle_label = np.random.randint(10)
                xs_cle_index = np.random.randint(len(mixup_pool_OL[xs_cle_label]))
                mixup_img_cle = (1 - lamdaOL) * mixup_pool_OL[xs_cle_label][xs_cle_index][0]
                mixup_img_batch[b] = mixup_img_cle.cpu().detach().numpy()

            mixup_img_batch = ep.from_numpy(ep.astensor(img_batch), mixup_img_batch).tensor +  + lamdaOL * img_batch
            pred_cle_mixup = classifier(mixup_img_batch)
            pred_cle_mixup_all_OL = pred_cle_mixup_all_OL + soft_max(pred_cle_mixup)

        pred_cle_mixup_all_OL = pred_cle_mixup_all_OL / num_sample_MIOL

        return pred_cle_mixup_all_OL

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class CombinedEnsembleModel(nn.Module):
    def __init__(self, combined_model):
        super(CombinedEnsembleModel, self).__init__()
        self.classifier = combined_model

    def forward(self, img_batch):
        return torch.stack([self.classifier.forward(img_batch) for _ in range(3)]).mean(axis=0)

In [8]:
combined_classifier = CombinedModel(classifier)
combined_classifier.eval();

combined_ensemble_classifier = CombinedEnsembleModel(combined_classifier)
combined_ensemble_classifier.eval();

In [9]:
iAT_model = fbn.models.PyTorchModel(classifier, bounds=(-1, 1), device=device)
iAT_OL_model = fbn.models.PyTorchModel(combined_classifier, bounds=(-1, 1), device=device)
iAT_OL_ensemble_model = fbn.models.PyTorchModel(combined_ensemble_classifier, bounds=(-1, 1), device=device)

In [10]:
for x_batch, y_batch in dataloader_test:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    break

### Oblivious attack

In [11]:
from tqdm import tqdm
t = tqdm(dataloader_test)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
acc = 0
total_samples = 0

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    #adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=4/255, num_steps=20)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_OL_model, x_batch, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'CLEAN (acc={acc / total_samples:.3f})')


CLEAN (acc=0.825):  11%|█▏        | 113/1000 [00:19<02:10,  6.78it/s]


CLEAN (acc=0.820):  22%|██▎       | 225/1000 [00:36<02:08,  6.03it/s]


CLEAN (acc=0.823):  34%|███▎      | 337/1000 [00:54<01:40,  6.57it/s]


CLEAN (acc=0.824):  45%|████▍     | 449/1000 [01:10<01:11,  7.68it/s]


CLEAN (acc=0.822):  56%|█████▌    | 561/1000 [01:25<01:02,  7.02it/s]


CLEAN (acc=0.823):  67%|██████▋   | 673/1000 [01:42<00:58,  5.60it/s]


CLEAN (acc=0.820):  78%|███████▊  | 785/1000 [01:59<00:33,  6.36it/s]


CLEAN (acc=0.822):  90%|████████▉ | 897/1000 [02:16<00:16,  6.38it/s]


CLEAN (acc=0.822): 100%|██████████| 1000/1000 [02:31<00:00,  6.59it/s]


In [13]:
acc = 0
total_samples = 0

t = tqdm(dataloader_test)
attack = fbn.attacks.PGD(iAT_model)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=4/255, num_steps=40)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_OL_model, adversarials, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')


ADV (acc=0.586):  12%|█▏        | 116/1000 [01:57<14:32,  1.01it/s]


ADV (acc=0.585):  22%|██▏       | 223/1000 [03:39<12:29,  1.04it/s]

KeyboardInterrupt: 

In [14]:
acc = 0
total_samples = 0

t = tqdm(dataloader_test)
attack = fbn.attacks.PGD(iAT_OL_model)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=1/255, num_steps=50)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_OL_model, adversarials, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')



  0%|          | 0/1000 [00:00<?, ?it/s]

ADV (acc=0.400):   0%|          | 0/1000 [00:18<?, ?it/s]

ADV (acc=0.400):   0%|          | 1/1000 [00:18<5:09:48, 18.61s/it]

ADV (acc=0.650):   0%|          | 1/1000 [00:37<5:09:48, 18.61s/it]

ADV (acc=0.650):   0%|          | 2/1000 [00:37<5:08:50, 18.57s/it]

ADV (acc=0.533):   0%|          | 2/1000 [00:55<5:08:50, 18.57s/it]

ADV (acc=0.533):   0%|          | 3/1000 [00:55<5:08:00, 18.54s/it]

ADV (acc=0.475):   0%|          | 3/1000 [01:13<5:08:00, 18.54s/it]

ADV (acc=0.475):   0%|          | 4/1000 [01:13<5:06:04, 18.44s/it]

ADV (acc=0.480):   0%|          | 4/1000 [01:32<5:06:04, 18.44s/it]

ADV (acc=0.480):   0%|          | 5/1000 [01:32<5:05:06, 18.40s/it]

ADV (acc=0.433):   0%|          | 5/1000 [01:50<5:05:06, 18.40s/it]

ADV (acc=0.433):   1%|          | 6/1000 [01:50<5:03:14, 18.30s/it]

ADV (acc=0.414):   1%|          | 6/1000 [02:08<5:03:14, 18.30s/it]

ADV (acc=0.414):   1%|          | 7/1000 [02:08<5:02:32, 18.28s/it]



KeyboardInterrupt: 

In [26]:
acc / total_samples

0.43980000651627776

In [27]:
total_samples

10000

In [11]:
acc = 0
total_samples = 0

t = tqdm(dataloader_test)
attack = fbn.attacks.PGD(iAT_OL_ensemble_model)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=1/255, num_steps=50)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_OL_ensemble_model, adversarials, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')


ADV (acc=0.440):  11%|█         | 110/1000 [1:31:31<12:39:54, 51.23s/it]


ADV (acc=0.442):  11%|█▏        | 113/1000 [1:34:04<12:37:32, 51.24s/it]

KeyboardInterrupt: 

In [ ]:
acc = 0
total_samples = 0

t = tqdm(dataloader_test)
attack = fbn.attacks.PGD(iAT_OL_model)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = 
    
    adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=2/255, num_steps=30)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_OL_model, adversarials, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')

In [24]:
attack = fbn.attacks.PGD(iAT_OL_model)
adversarials = []

for k in range(10):
    _adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=2/255, num_steps=30)  # L-inf norm
    adversarials.append(_adversarials)
    acc = fbn.utils.accuracy(iAT_OL_model, _adversarials, y_batch)

    repeated_results = []
    for _ in range(100):
        repeated_results.append((iAT_OL_model.forward(_adversarials).argmax(1) == y_batch).cpu().numpy())

    print(f'(k, {acc:.3f}) {np.stack(repeated_results).mean(0)}')

(k, 0.300) [0.   0.   0.32 0.04 0.   0.84 0.01 0.   0.   1.  ]
(k, 0.300) [0.   0.   0.2  0.11 0.   0.71 0.   0.   0.   1.  ]
(k, 0.300) [0.   0.   0.2  0.15 0.   0.75 0.   0.   0.   1.  ]
(k, 0.200) [0.   0.   0.27 0.1  0.   0.66 0.   0.   0.   1.  ]
(k, 0.200) [0.   0.   0.24 0.12 0.   0.79 0.   0.   0.   1.  ]


In [25]:
attack = fbn.attacks.PGD(iAT_OL_model)
adversarials = []

for k in range(10):
    _adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=3/255, num_steps=30)  # L-inf norm
    adversarials.append(_adversarials)
    acc = fbn.utils.accuracy(iAT_OL_model, _adversarials, y_batch)

    repeated_results = []
    for _ in range(100):
        repeated_results.append((iAT_OL_model.forward(_adversarials).argmax(1) == y_batch).cpu().numpy())

    print(f'(k, {acc:.3f}) {np.stack(repeated_results).mean(0)}')

(k, 0.100) [0.   0.   0.28 0.12 0.   0.8  0.   0.   0.   1.  ]
(k, 0.300) [0.   0.   0.29 0.11 0.   0.77 0.   0.   0.   1.  ]
(k, 0.200) [0.   0.   0.17 0.17 0.   0.74 0.02 0.   0.   1.  ]
(k, 0.200) [0.   0.   0.19 0.1  0.   0.82 0.   0.   0.   1.  ]
(k, 0.300) [0.   0.   0.25 0.09 0.   0.74 0.01 0.   0.   1.  ]
(k, 0.200) [0.   0.   0.21 0.09 0.   0.76 0.   0.   0.   1.  ]
(k, 0.300) [0.   0.   0.28 0.06 0.   0.76 0.   0.   0.   1.  ]
(k, 0.200) [0.   0.   0.22 0.08 0.   0.7  0.   0.   0.   1.  ]
(k, 0.400) [0.   0.   0.16 0.04 0.   0.77 0.   0.   0.   1.  ]
(k, 0.200) [0.   0.   0.27 0.06 0.   0.72 0.   0.   0.   1.  ]


In [28]:
attack = fbn.attacks.PGD(iAT_OL_model)
adversarials = []
best_adversarials = [None] * x_batch.shape[0]
best_probs = [np.inf] * x_batch.shape[0]

for k in range(5):
    _adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=3/255, num_steps=30)  # L-inf norm
    adversarials.append(_adversarials)
    acc = fbn.utils.accuracy(iAT_OL_model, _adversarials, y_batch)

    repeated_results = []
    for _ in range(100):
        repeated_results.append((iAT_OL_model.forward(_adversarials).argmax(1) == y_batch).cpu().numpy())

    probs = np.stack(repeated_results).mean(0)
        
    for c in range(x_batch.shape[0]):
        if probs[c] < best_probs[c]:
            best_probs[c] = probs[c]
            best_adversarials[c] = _adversarials[c]
        
    print(f'(k, {acc:.3f}) {np.stack(repeated_results).mean(0)}')
    best_adversarials = torch.stack(best_adversarials)

(k, 0.100) [0.   0.   0.22 0.09 0.   0.79 0.   0.   0.   1.  ]
(k, 0.200) [0.   0.   0.22 0.15 0.   0.75 0.   0.   0.   1.  ]
(k, 0.300) [0.   0.   0.37 0.11 0.   0.69 0.01 0.   0.   1.  ]
(k, 0.300) [0.   0.   0.2  0.07 0.   0.7  0.   0.   0.   1.  ]
(k, 0.300) [0.   0.   0.23 0.07 0.   0.73 0.   0.   0.   1.  ]


In [ ]:
attack = fbn.attacks.PGD(iAT_OL_model)

acc = 0
total_samples = 0
t = tqdm(dataloader_test)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = []
    best_adversarials = [None] * x_batch.shape[0]
    best_probs = [np.inf] * x_batch.shape[0]

    for k in range(5):
        _adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=3/255, num_steps=30)  # L-inf norm
        adversarials.append(_adversarials)

        repeated_results = []
        for _ in range(100):
            repeated_results.append((iAT_OL_model.forward(_adversarials).argmax(1) == y_batch).cpu().numpy())

        probs = np.stack(repeated_results).mean(0)

        for c in range(x_batch.shape[0]):
            if probs[c] < best_probs[c]:
                best_probs[c] = probs[c]
                best_adversarials[c] = _adversarials[c]

    adversarials = torch.stack(best_adversarials)
    
    batch_acc = fbn.utils.accuracy(iAT_OL_model, adversarials, y_batch)
    acc += batch_acc * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')






  0%|          | 0/1000 [00:00<?, ?it/s]




ADV (acc=0.100):   0%|          | 0/1000 [02:00<?, ?it/s]




ADV (acc=0.100):   0%|          | 1/1000 [02:00<33:27:37, 120.58s/it]




ADV (acc=0.450):   0%|          | 1/1000 [04:00<33:27:37, 120.58s/it]




ADV (acc=0.450):   0%|          | 2/1000 [04:00<33:24:46, 120.53s/it]




ADV (acc=0.433):   0%|          | 2/1000 [06:01<33:24:46, 120.53s/it]




ADV (acc=0.433):   0%|          | 3/1000 [06:01<33:21:27, 120.45s/it]




ADV (acc=0.425):   0%|          | 3/1000 [08:01<33:21:27, 120.45s/it]




ADV (acc=0.425):   0%|          | 4/1000 [08:01<33:17:20, 120.32s/it]




ADV (acc=0.440):   0%|          | 4/1000 [09:59<33:17:20, 120.32s/it]




ADV (acc=0.440):   0%|          | 5/1000 [09:59<33:07:21, 119.84s/it]




ADV (acc=0.400):   0%|          | 5/1000 [11:59<33:07:21, 119.84s/it]




ADV (acc=0.400):   1%|          | 6/1000 [11:59<33:01:55, 119.63s/it]




ADV (acc=0.400):   1%|          | 6/1000 [13:59<33:01:55, 119.63s/it]






In [52]:
acc

0.800000011920929

In [53]:
total_samples

10

In [32]:
best_adversarials = torch.stack(best_adversarials)

In [49]:
fbn.utils.accuracy(iAT_OL_model, best_adversarials, y_batch)

0.30000001192092896

In [31]:
_adversarials.shape

torch.Size([10, 3, 32, 32])

In [19]:
repeated_results = []
for _ in range(5):
    repeated_results.append((iAT_OL_model.forward(adversarials).argmax(1) == y_batch).cpu().numpy())

In [21]:
np.stack(repeated_results).mean(0)

array([0. , 0. , 0. , 1. , 0. , 1. , 0. , 0. , 0.4, 0. ])

In [16]:
(iAT_OL_model.forward(adversarials).argmax(1) == y_batch).cpu().numpy()

array([False, False, False, False, False,  True, False, False, False,
       False])

In [15]:
acc = 0
total_samples = 0

t = tqdm(dataloader_test)
attack = fbn.attacks.PGD(iAT_model)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=4/255, num_steps=20)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_model, adversarials, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')




  0%|          | 0/1000 [00:00<?, ?it/s]


ADV (acc=0.000):   0%|          | 0/1000 [00:00<?, ?it/s]


ADV (acc=0.000):   0%|          | 1/1000 [00:00<13:55,  1.20it/s]


ADV (acc=0.300):   0%|          | 1/1000 [00:01<13:55,  1.20it/s]


ADV (acc=0.300):   0%|          | 2/1000 [00:01<11:54,  1.40it/s]


ADV (acc=0.333):   0%|          | 2/1000 [00:01<11:54,  1.40it/s]


ADV (acc=0.333):   0%|          | 3/1000 [00:01<10:30,  1.58it/s]


ADV (acc=0.300):   0%|          | 3/1000 [00:02<10:30,  1.58it/s]


ADV (acc=0.300):   0%|          | 4/1000 [00:02<09:28,  1.75it/s]


ADV (acc=0.320):   0%|          | 4/1000 [00:02<09:28,  1.75it/s]


ADV (acc=0.320):   0%|          | 5/1000 [00:02<08:38,  1.92it/s]


ADV (acc=0.283):   0%|          | 5/1000 [00:02<08:38,  1.92it/s]


ADV (acc=0.283):   1%|          | 6/1000 [00:02<07:47,  2.13it/s]


ADV (acc=0.286):   1%|          | 6/1000 [00:03<07:47,  2.13it/s]


ADV (acc=0.286):   1%|          | 7/1000 [00:03<07:14,  2.28it/s]


ADV (acc=0

ADV (acc=0.382):  10%|█         | 104/1000 [00:39<05:49,  2.57it/s]


ADV (acc=0.382):  10%|█         | 105/1000 [00:39<05:54,  2.52it/s]


ADV (acc=0.382):  10%|█         | 105/1000 [00:40<05:54,  2.52it/s]


ADV (acc=0.382):  11%|█         | 106/1000 [00:40<06:08,  2.42it/s]


ADV (acc=0.383):  11%|█         | 106/1000 [00:40<06:08,  2.42it/s]


ADV (acc=0.383):  11%|█         | 107/1000 [00:40<06:18,  2.36it/s]


ADV (acc=0.384):  11%|█         | 107/1000 [00:41<06:18,  2.36it/s]


ADV (acc=0.384):  11%|█         | 108/1000 [00:41<06:33,  2.26it/s]


ADV (acc=0.385):  11%|█         | 108/1000 [00:41<06:33,  2.26it/s]


ADV (acc=0.385):  11%|█         | 109/1000 [00:41<06:38,  2.24it/s]


ADV (acc=0.385):  11%|█         | 109/1000 [00:42<06:38,  2.24it/s]


ADV (acc=0.385):  11%|█         | 110/1000 [00:42<06:38,  2.23it/s]


ADV (acc=0.387):  11%|█         | 110/1000 [00:42<06:38,  2.23it/s]


ADV (acc=0.387):  11%|█         | 111/1000 [00:42<06:53,  2.15it/s]


ADV (acc=0.388):  11

ADV (acc=0.377):  21%|██        | 207/1000 [01:17<04:25,  2.98it/s]


ADV (acc=0.377):  21%|██        | 208/1000 [01:17<04:21,  3.03it/s]


ADV (acc=0.378):  21%|██        | 208/1000 [01:18<04:21,  3.03it/s]


ADV (acc=0.378):  21%|██        | 209/1000 [01:18<04:17,  3.07it/s]


ADV (acc=0.379):  21%|██        | 209/1000 [01:18<04:17,  3.07it/s]


ADV (acc=0.379):  21%|██        | 210/1000 [01:18<04:15,  3.09it/s]


ADV (acc=0.379):  21%|██        | 210/1000 [01:18<04:15,  3.09it/s]


ADV (acc=0.379):  21%|██        | 211/1000 [01:18<04:14,  3.10it/s]


ADV (acc=0.378):  21%|██        | 211/1000 [01:19<04:14,  3.10it/s]


ADV (acc=0.378):  21%|██        | 212/1000 [01:19<04:23,  2.99it/s]


ADV (acc=0.377):  21%|██        | 212/1000 [01:19<04:23,  2.99it/s]


ADV (acc=0.377):  21%|██▏       | 213/1000 [01:19<04:55,  2.66it/s]


ADV (acc=0.378):  21%|██▏       | 213/1000 [01:19<04:55,  2.66it/s]


ADV (acc=0.378):  21%|██▏       | 214/1000 [01:19<05:03,  2.59it/s]


ADV (acc=0.380):  21

ADV (acc=0.377):  31%|███       | 310/1000 [01:54<03:50,  2.99it/s]


ADV (acc=0.377):  31%|███       | 311/1000 [01:54<03:54,  2.93it/s]


ADV (acc=0.378):  31%|███       | 311/1000 [01:55<03:54,  2.93it/s]


ADV (acc=0.378):  31%|███       | 312/1000 [01:55<03:50,  2.98it/s]


ADV (acc=0.378):  31%|███       | 312/1000 [01:55<03:50,  2.98it/s]


ADV (acc=0.378):  31%|███▏      | 313/1000 [01:55<03:44,  3.06it/s]


ADV (acc=0.377):  31%|███▏      | 313/1000 [01:55<03:44,  3.06it/s]


ADV (acc=0.377):  31%|███▏      | 314/1000 [01:55<03:41,  3.10it/s]


ADV (acc=0.377):  31%|███▏      | 314/1000 [01:56<03:41,  3.10it/s]


ADV (acc=0.377):  32%|███▏      | 315/1000 [01:56<03:51,  2.96it/s]


ADV (acc=0.378):  32%|███▏      | 315/1000 [01:56<03:51,  2.96it/s]


ADV (acc=0.378):  32%|███▏      | 316/1000 [01:56<04:08,  2.75it/s]


ADV (acc=0.379):  32%|███▏      | 316/1000 [01:56<04:08,  2.75it/s]


ADV (acc=0.379):  32%|███▏      | 317/1000 [01:56<04:20,  2.63it/s]


ADV (acc=0.378):  32

ADV (acc=0.379):  41%|████▏     | 413/1000 [02:31<03:15,  3.00it/s]


ADV (acc=0.379):  41%|████▏     | 414/1000 [02:31<03:11,  3.05it/s]


ADV (acc=0.380):  41%|████▏     | 414/1000 [02:31<03:11,  3.05it/s]


ADV (acc=0.380):  42%|████▏     | 415/1000 [02:31<03:09,  3.08it/s]


ADV (acc=0.380):  42%|████▏     | 415/1000 [02:31<03:09,  3.08it/s]


ADV (acc=0.380):  42%|████▏     | 416/1000 [02:31<03:07,  3.11it/s]


ADV (acc=0.379):  42%|████▏     | 416/1000 [02:32<03:07,  3.11it/s]


ADV (acc=0.379):  42%|████▏     | 417/1000 [02:32<03:05,  3.14it/s]


ADV (acc=0.379):  42%|████▏     | 417/1000 [02:32<03:05,  3.14it/s]


ADV (acc=0.379):  42%|████▏     | 418/1000 [02:32<03:05,  3.14it/s]


ADV (acc=0.380):  42%|████▏     | 418/1000 [02:32<03:05,  3.14it/s]


ADV (acc=0.380):  42%|████▏     | 419/1000 [02:32<03:03,  3.16it/s]


ADV (acc=0.380):  42%|████▏     | 419/1000 [02:32<03:03,  3.16it/s]


ADV (acc=0.380):  42%|████▏     | 420/1000 [02:32<03:02,  3.18it/s]


ADV (acc=0.380):  42

ADV (acc=0.378):  52%|█████▏    | 516/1000 [03:09<02:58,  2.71it/s]


ADV (acc=0.378):  52%|█████▏    | 517/1000 [03:09<02:49,  2.85it/s]


ADV (acc=0.378):  52%|█████▏    | 517/1000 [03:09<02:49,  2.85it/s]


ADV (acc=0.378):  52%|█████▏    | 518/1000 [03:09<02:44,  2.93it/s]


ADV (acc=0.377):  52%|█████▏    | 518/1000 [03:09<02:44,  2.93it/s]


ADV (acc=0.377):  52%|█████▏    | 519/1000 [03:09<02:41,  2.97it/s]


ADV (acc=0.377):  52%|█████▏    | 519/1000 [03:09<02:41,  2.97it/s]


ADV (acc=0.377):  52%|█████▏    | 520/1000 [03:09<02:39,  3.01it/s]


ADV (acc=0.377):  52%|█████▏    | 520/1000 [03:10<02:39,  3.01it/s]


ADV (acc=0.377):  52%|█████▏    | 521/1000 [03:10<02:35,  3.08it/s]


ADV (acc=0.377):  52%|█████▏    | 521/1000 [03:10<02:35,  3.08it/s]


ADV (acc=0.377):  52%|█████▏    | 522/1000 [03:10<02:43,  2.92it/s]


ADV (acc=0.377):  52%|█████▏    | 522/1000 [03:11<02:43,  2.92it/s]


ADV (acc=0.377):  52%|█████▏    | 523/1000 [03:11<02:46,  2.86it/s]


ADV (acc=0.377):  52

ADV (acc=0.375):  62%|██████▏   | 619/1000 [03:45<02:02,  3.11it/s]


ADV (acc=0.375):  62%|██████▏   | 620/1000 [03:45<02:01,  3.12it/s]


ADV (acc=0.375):  62%|██████▏   | 620/1000 [03:45<02:01,  3.12it/s]


ADV (acc=0.375):  62%|██████▏   | 621/1000 [03:45<02:00,  3.13it/s]


ADV (acc=0.375):  62%|██████▏   | 621/1000 [03:46<02:00,  3.13it/s]


ADV (acc=0.375):  62%|██████▏   | 622/1000 [03:46<02:03,  3.07it/s]


ADV (acc=0.375):  62%|██████▏   | 622/1000 [03:46<02:03,  3.07it/s]


ADV (acc=0.375):  62%|██████▏   | 623/1000 [03:46<02:07,  2.95it/s]


ADV (acc=0.375):  62%|██████▏   | 623/1000 [03:47<02:07,  2.95it/s]


ADV (acc=0.375):  62%|██████▏   | 624/1000 [03:47<02:09,  2.91it/s]


ADV (acc=0.376):  62%|██████▏   | 624/1000 [03:47<02:09,  2.91it/s]


ADV (acc=0.376):  62%|██████▎   | 625/1000 [03:47<02:05,  2.99it/s]


ADV (acc=0.376):  62%|██████▎   | 625/1000 [03:47<02:05,  2.99it/s]


ADV (acc=0.376):  63%|██████▎   | 626/1000 [03:47<02:09,  2.89it/s]


ADV (acc=0.376):  63

ADV (acc=0.373):  72%|███████▏  | 722/1000 [04:21<01:30,  3.06it/s]


ADV (acc=0.373):  72%|███████▏  | 723/1000 [04:21<01:29,  3.10it/s]


ADV (acc=0.373):  72%|███████▏  | 723/1000 [04:21<01:29,  3.10it/s]


ADV (acc=0.373):  72%|███████▏  | 724/1000 [04:21<01:32,  2.98it/s]


ADV (acc=0.373):  72%|███████▏  | 724/1000 [04:22<01:32,  2.98it/s]


ADV (acc=0.373):  72%|███████▎  | 725/1000 [04:22<01:34,  2.92it/s]


ADV (acc=0.373):  72%|███████▎  | 725/1000 [04:22<01:34,  2.92it/s]


ADV (acc=0.373):  73%|███████▎  | 726/1000 [04:22<01:35,  2.88it/s]


ADV (acc=0.373):  73%|███████▎  | 726/1000 [04:22<01:35,  2.88it/s]


ADV (acc=0.373):  73%|███████▎  | 727/1000 [04:22<01:32,  2.97it/s]


ADV (acc=0.373):  73%|███████▎  | 727/1000 [04:23<01:32,  2.97it/s]


ADV (acc=0.373):  73%|███████▎  | 728/1000 [04:23<01:29,  3.03it/s]


ADV (acc=0.373):  73%|███████▎  | 728/1000 [04:23<01:29,  3.03it/s]


ADV (acc=0.373):  73%|███████▎  | 729/1000 [04:23<01:27,  3.09it/s]


ADV (acc=0.373):  73

ADV (acc=0.373):  82%|████████▎ | 825/1000 [04:56<01:06,  2.65it/s]


ADV (acc=0.373):  83%|████████▎ | 826/1000 [04:56<01:03,  2.76it/s]


ADV (acc=0.373):  83%|████████▎ | 826/1000 [04:56<01:03,  2.76it/s]


ADV (acc=0.373):  83%|████████▎ | 827/1000 [04:56<01:04,  2.70it/s]


ADV (acc=0.373):  83%|████████▎ | 827/1000 [04:56<01:04,  2.70it/s]


ADV (acc=0.373):  83%|████████▎ | 828/1000 [04:56<01:03,  2.69it/s]


ADV (acc=0.372):  83%|████████▎ | 828/1000 [04:57<01:03,  2.69it/s]


ADV (acc=0.372):  83%|████████▎ | 829/1000 [04:57<01:01,  2.79it/s]


ADV (acc=0.372):  83%|████████▎ | 829/1000 [04:57<01:01,  2.79it/s]


ADV (acc=0.372):  83%|████████▎ | 830/1000 [04:57<00:58,  2.90it/s]


ADV (acc=0.372):  83%|████████▎ | 830/1000 [04:57<00:58,  2.90it/s]


ADV (acc=0.372):  83%|████████▎ | 831/1000 [04:57<00:56,  2.97it/s]


ADV (acc=0.372):  83%|████████▎ | 831/1000 [04:58<00:56,  2.97it/s]


ADV (acc=0.372):  83%|████████▎ | 832/1000 [04:58<00:55,  3.03it/s]


ADV (acc=0.372):  83

ADV (acc=0.371):  93%|█████████▎| 928/1000 [05:30<00:24,  2.95it/s]


ADV (acc=0.371):  93%|█████████▎| 929/1000 [05:30<00:23,  3.02it/s]


ADV (acc=0.371):  93%|█████████▎| 929/1000 [05:30<00:23,  3.02it/s]


ADV (acc=0.371):  93%|█████████▎| 930/1000 [05:30<00:23,  3.04it/s]


ADV (acc=0.371):  93%|█████████▎| 930/1000 [05:30<00:23,  3.04it/s]


ADV (acc=0.371):  93%|█████████▎| 931/1000 [05:30<00:22,  3.10it/s]


ADV (acc=0.371):  93%|█████████▎| 931/1000 [05:31<00:22,  3.10it/s]


ADV (acc=0.371):  93%|█████████▎| 932/1000 [05:31<00:21,  3.14it/s]


ADV (acc=0.371):  93%|█████████▎| 932/1000 [05:31<00:21,  3.14it/s]


ADV (acc=0.371):  93%|█████████▎| 933/1000 [05:31<00:21,  3.17it/s]


ADV (acc=0.371):  93%|█████████▎| 933/1000 [05:31<00:21,  3.17it/s]


ADV (acc=0.371):  93%|█████████▎| 934/1000 [05:31<00:20,  3.17it/s]


ADV (acc=0.370):  93%|█████████▎| 934/1000 [05:32<00:20,  3.17it/s]


ADV (acc=0.370):  94%|█████████▎| 935/1000 [05:32<00:20,  3.19it/s]


ADV (acc=0.371):  94

In [16]:
acc = 0
total_samples = 0

t = tqdm(dataloader_test)
attack = fbn.attacks.PGD(iAT_model)

for x_batch, y_batch in t:
    x_batch = x_batch.to(device)
    y_batch = y_batch.argmax(1).to(device)
    
    adversarials = attack(x_batch, y_batch, epsilon=16/255, step_size=3/255, num_steps=50)  # L-inf norm
    acc += fbn.utils.accuracy(iAT_model, adversarials, y_batch) * x_batch.shape[0]
    total_samples += x_batch.shape[0]
    
    t.set_description(f'ADV (acc={acc / total_samples:.3f})')




  0%|          | 0/1000 [00:00<?, ?it/s]


ADV (acc=0.000):   0%|          | 0/1000 [00:01<?, ?it/s]


ADV (acc=0.000):   0%|          | 1/1000 [00:01<22:17,  1.34s/it]


ADV (acc=0.300):   0%|          | 1/1000 [00:02<22:17,  1.34s/it]


ADV (acc=0.300):   0%|          | 2/1000 [00:02<19:34,  1.18s/it]


ADV (acc=0.333):   0%|          | 2/1000 [00:03<19:34,  1.18s/it]


ADV (acc=0.333):   0%|          | 3/1000 [00:03<18:49,  1.13s/it]


ADV (acc=0.325):   0%|          | 3/1000 [00:04<18:49,  1.13s/it]


ADV (acc=0.325):   0%|          | 4/1000 [00:04<18:07,  1.09s/it]


ADV (acc=0.340):   0%|          | 4/1000 [00:05<18:07,  1.09s/it]


ADV (acc=0.340):   0%|          | 5/1000 [00:05<17:30,  1.06s/it]


ADV (acc=0.300):   0%|          | 5/1000 [00:06<17:30,  1.06s/it]


ADV (acc=0.300):   1%|          | 6/1000 [00:06<17:16,  1.04s/it]


ADV (acc=0.300):   1%|          | 6/1000 [00:07<17:16,  1.04s/it]


ADV (acc=0.300):   1%|          | 7/1000 [00:07<17:19,  1.05s/it]


ADV (acc=0

ADV (acc=0.371):  10%|█         | 104/1000 [01:33<13:19,  1.12it/s]


ADV (acc=0.371):  10%|█         | 105/1000 [01:33<13:11,  1.13it/s]


ADV (acc=0.371):  10%|█         | 105/1000 [01:34<13:11,  1.13it/s]


ADV (acc=0.371):  11%|█         | 106/1000 [01:34<13:27,  1.11it/s]


ADV (acc=0.372):  11%|█         | 106/1000 [01:35<13:27,  1.11it/s]


ADV (acc=0.372):  11%|█         | 107/1000 [01:35<13:04,  1.14it/s]


ADV (acc=0.373):  11%|█         | 107/1000 [01:36<13:04,  1.14it/s]


ADV (acc=0.373):  11%|█         | 108/1000 [01:36<12:50,  1.16it/s]


ADV (acc=0.374):  11%|█         | 108/1000 [01:37<12:50,  1.16it/s]


ADV (acc=0.374):  11%|█         | 109/1000 [01:37<12:44,  1.17it/s]


ADV (acc=0.375):  11%|█         | 109/1000 [01:37<12:44,  1.17it/s]


ADV (acc=0.375):  11%|█         | 110/1000 [01:38<12:21,  1.20it/s]


ADV (acc=0.377):  11%|█         | 110/1000 [01:38<12:21,  1.20it/s]


ADV (acc=0.377):  11%|█         | 111/1000 [01:38<12:09,  1.22it/s]


ADV (acc=0.378):  11

ADV (acc=0.367):  21%|██        | 207/1000 [02:57<10:43,  1.23it/s]


ADV (acc=0.367):  21%|██        | 208/1000 [02:57<11:05,  1.19it/s]


ADV (acc=0.367):  21%|██        | 208/1000 [02:58<11:05,  1.19it/s]


ADV (acc=0.367):  21%|██        | 209/1000 [02:58<11:29,  1.15it/s]


ADV (acc=0.369):  21%|██        | 209/1000 [02:58<11:29,  1.15it/s]


ADV (acc=0.369):  21%|██        | 210/1000 [02:58<11:08,  1.18it/s]


ADV (acc=0.368):  21%|██        | 210/1000 [02:59<11:08,  1.18it/s]


ADV (acc=0.368):  21%|██        | 211/1000 [02:59<10:51,  1.21it/s]


ADV (acc=0.367):  21%|██        | 211/1000 [03:00<10:51,  1.21it/s]


ADV (acc=0.367):  21%|██        | 212/1000 [03:00<11:43,  1.12it/s]


ADV (acc=0.367):  21%|██        | 212/1000 [03:01<11:43,  1.12it/s]


ADV (acc=0.367):  21%|██▏       | 213/1000 [03:01<11:16,  1.16it/s]


ADV (acc=0.368):  21%|██▏       | 213/1000 [03:02<11:16,  1.16it/s]


ADV (acc=0.368):  21%|██▏       | 214/1000 [03:02<10:57,  1.19it/s]


ADV (acc=0.369):  21

KeyboardInterrupt: 